# Introduction
Group members(& emails?):

# Table of contents
TODO: titles with internal links to move there

# Decomposition and breakdown

blah blah

# Analysis of data given (records.csv)

# Planning and design of algorithm
insert diagram drawn by Cherry here

## Flowchart and pseudocode

lorem ipsum

# Excel things

In [3]:
'''
file = original excel file
rawFileData = giant list where each item is a row converted into a single string
fileHeader = first row "Tutorial Group,Student ID,School,Name,Gender,CGPA"
fileData = each row in the file turned into a list of cells as strings
studentData = list of dictionaries, each representing a student's data ('student ID', 'TG', 'School', 'Name', 'Gender', 'CGPA')
'''

'\nfile = original excel file\nrawFileData = giant list where each item is a row converted into a single string\nfileHeader = first row "Tutorial Group,Student ID,School,Name,Gender,CGPA"\nfileData = each row in the file turned into a list of cells as strings\nstudentData = list of dictionaries, each representing a student\'s data (\'student ID\', \'TG\', \'School\', \'Name\', \'Gender\', \'CGPA\')\n'

In [4]:
file = open("records.csv", "r")
rawFileData = file.readlines()
fileData = []
for i in rawFileData:
    i = i.split(",")
    i[5] = i[5].replace('\n', '')
    fileData.append(i)
fileHeader = fileData.pop(0) #list of cells from first line

print(fileData[0])

['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02']


In [5]:
#global variables


In [6]:
def initStudents():      #creates studentData, a list of dictionaries, each representing a student
    studentData = []
    studentCount = 0
    for row in fileData:
        studentCount += 1
        if row[4] == 'Male':
            gender = True
        elif row[4] == 'Female':
            gender = False
        studentData.append({'ID': int(row[1]),'TG': int(row[0][2:]), 'School': row[2], 'Name': row[3], 'Gender': gender, 'CGPA': float(row[5])})
    return studentData

In [7]:
studentData = initStudents()
print(studentData[0])
print(len(studentData))

{'ID': 5002, 'TG': 1, 'School': 'CCDS', 'Name': 'Aarav Singh', 'Gender': True, 'CGPA': 4.02}
6000


# Prep for Sorting

In [8]:
# Apply function separately to individual tutorial groups
def sortingPrep (studentData):
    # Sort list in descending order of CGPA
    studentData.sort(key=lambda x: x['CGPA'], reverse=True)

    # Split list into buckets
    partition = [studentData[i:i + n] for i in range(0, len(studentData), n)]

    # for index, bucket in enumerate(partition, start=1):
    #     print(f"\nBucket {index} ({len(bucket)}):")
    #     for s in bucket:
    #         print(f"  {s['Name']}, {s['CGPA']}, TG{s['TG']}")

    return partition

In [9]:
def distribute(studentData, n):

    buckets = sortingPrep(studentData)

    total = len(studentData)
    numGrp = total // n
    r = total % n
    grpSizes = [n + 1 if i < r else n for i in range(numGrp)]
    grps = [[] for _ in range(numGrp)]

    # Calculate school & gender ratio
    schoolCnt = {}
    for s in studentData:
        school = s['School']
        schoolCnt[school] = schoolCnt.get(school, 0) + 1
    sortedSchools = sorted(schoolCnt.keys(), key=lambda x: schoolCnt[x], reverse=True)

    totalMale = sum(1 for s in studentData if s['Gender'])
    # mRatio = totalMale / total
    mMin = totalMale // numGrp
    mRemainder = totalMale % numGrp
    idealMale = [mMin + (1 if i < mRemainder else 0) for i in range(len(grpSizes))]

    totalFemale = len(studentData) - totalMale
    fMin = totalFemale // numGrp
    fRemainder = totalFemale % numGrp
    idealFemale = [fMin + (1 if i < fRemainder else 0) for i in range(len(grpSizes))]
    #     idealMale = [math.ceil(mRatio * size) for size in grpSizes]
    #     idealFemale = [size - idealMale[i] for i, size in enumerate(grpSizes)]
    currMale = [0] * numGrp
    currFemale = [0] * numGrp

    sortedStudents = sorted(
        studentData,
        key=lambda s: (sortedSchools.index(s['School']), -s['CGPA'])
    )

    # Distribution
    grpIndex = 0
    for student in sortedStudents:
        # try to find a group that still needs this gender
        for offset in range(numGrp):
            g = (grpIndex + offset) % numGrp
            if len(grps[g]) < grpSizes[g]:
                if student['Gender'] and currMale[g] < idealMale[g]:
                    grps[g].append(student)
                    currMale[g] += 1
                    # print(f"append {student} to grp {g}")
                    break
                elif not student['Gender'] and currFemale[g] < idealFemale[g]:
                    grps[g].append(student)
                    currFemale[g] += 1
                    # print(f"append {student} to grp {g}")
                    break
        else:
            # if all gender quotas full, put in group with least size
            g = min(range(numGrp), key=lambda x: len(grps[x]))
            grps[g].append(student)
            if student['Gender']:
                currMale[g] += 1
            else:
                currFemale[g] += 1
            # print(f"append {student} to grp {g} (backup)")

        grpIndex += 1

    return grps

In [44]:
n = 7

# Apply to each TG 
all_tgs = sorted(set(s['TG'] for s in studentData))
for tg in all_tgs:
    tgStuds = [s for s in studentData if s['TG'] == tg]
    totalStuds = len(tgStuds)

    # Get TG gender ratio
    mCount_tg = sum(1 for s in tgStuds if s['Gender'])
    fCount_tg = totalStuds - mCount_tg
    mRatio_tg = mCount_tg / totalStuds if totalStuds else 0
    fRatio_tg = 1 - mRatio_tg

    print(f"\nTutorial Group {tg}")
    print(f"Total students: {totalStuds}")
    print(f"Gender ratio: {mCount_tg}M / {fCount_tg}F "
          f"({mRatio_tg:.2%} male, {fRatio_tg:.2%} female)")

    # Get TG school ratio
    schoolDistTG = {}
    for s in tgStuds:
        schoolDistTG[s['School']] = schoolDistTG.get(s['School'], 0) + 1
    sortedSchoolsTG = sorted(schoolDistTG.keys(), key=lambda x: schoolDistTG[x], reverse=True)
    schoolSummary = ", ".join(f"{sch}: {schoolDistTG[sch]}" for sch in sortedSchoolsTG)
    print(f"School distribution: {schoolSummary}")

    # Distribute
    finalGrps = distribute(tgStuds, n)

    # Print results per grp
    totalInGrps = 0
    for i, g in enumerate(finalGrps, start=1):
        mCount = sum(1 for s in g if s['Gender'])
        fCount = len(g) - mCount
        totalInGrps += len(g)
        print(f"\nGroup {i} (size {len(g)}): Males: {mCount}, Females: {fCount}")
        for s in g:
            print(f"  - {s['Name']} ({s['School']}, {'M' if s['Gender'] else 'F'}, CGPA: {s['CGPA']})")

    print(f"\nTotal assigned in TG{tg}: {totalInGrps} / {totalStuds}")



Tutorial Group 1
Total students: 50
Gender ratio: 21M / 29F (42.00% male, 58.00% female)
School distribution: EEE: 10, CoB (NBS): 8, CCDS: 5, SoH: 5, SSS: 5, MAE: 4, CoE: 3, WKW SCI: 2, SPMS: 2, SBS: 1, ASE: 1, NIE: 1, ADM: 1, CCEB: 1, MSE: 1

Group 1 (size 8): Males: 3, Females: 5
  - Adlan Bin Rahman (EEE, M, CGPA: 4.2)
  - Jie Zhang (EEE, M, CGPA: 4.03)
  - Felix Yip (CoB (NBS), M, CGPA: 4.08)
  - Lila Patel (SSS, F, CGPA: 4.07)
  - Amelia Kim (CCDS, F, CGPA: 4.11)
  - Zara Chang (MSE, F, CGPA: 4.11)
  - Nhung Vu (ADM, F, CGPA: 4.04)
  - Karen Wong (SBS, F, CGPA: 3.95)

Group 2 (size 7): Males: 3, Females: 4
  - Xun Wei (EEE, F, CGPA: 4.18)
  - Mei Hong (EEE, F, CGPA: 4.02)
  - Isabella Thompson (CoB (NBS), F, CGPA: 4.03)
  - Truong Minh Chau (SSS, F, CGPA: 3.88)
  - Lucas Walker (MAE, M, CGPA: 4.06)
  - Anthony Liu (WKW SCI, M, CGPA: 4.15)
  - Nguyen Van Sam (NIE, M, CGPA: 3.85)

Group 3 (size 7): Males: 3, Females: 4
  - Henry Foster (EEE, M, CGPA: 4.11)
  - Gabriel Young (EEE, M

# Getting it back into an excel file and things

In [11]:
testfile = open("testrecords.csv", "w")
fileHeader = fileHeader
fileData.insert(0, fileHeader) #puts back header line
fileData[1].append('')
fileData[1].append('AAAAAAAH')
print(fileData[0])
print(fileData[1])
for i in range(len(fileData)):  #for each list, join items by ','. Then prints each line into new excel sheet
    fileData[i] = ','.join(fileData[i])
    print(fileData[i], file = testfile, end = '\n')

['Tutorial Group', 'Student ID', 'School', 'Name', 'Gender', 'CGPA']
['G-1', '5002', 'CCDS', 'Aarav Singh', 'Male', '4.02', '', 'AAAAAAAH']


# Algorithm Evaluation

## Basic Factors
No majority of students of the same following factors:
1. School
2. Gender
3. Current CGPA

General flow:
TODO: insert flowchart
- For each tutorial group, check for a single factor first, to find the group with the least diversity
- Then rank them to see with tutorial group has a group with the least diversity

### Finding the diversity of a single factor of a single group
Required data: list of students in the group, required factor of the student
<br>
Output: The number for diversity
<br>
Start with finding the number of occurences of each factor

In [12]:
# Parameters:
#  factor: string of factor to check, same as the key in the dictionary
#  groupData: data needed as a list of dictionaries
# Return:
#  occurences: dictionary of {factor1: numOccurences, factor2: numOccurences ... } as string:int
def FindOccurencesInGroup(factor, groupData : list) -> dict:
    size = len(groupData)
    occurences = {} # Dictionary to keep track of occurences of each factor
    for i in groupData:
        factorOfStudent = i[factor]
        # If factor already exists in occurences, add 1 to it, otherwise create it and set to 1
        if factorOfStudent in occurences.keys():
            occurences[factorOfStudent] += 1
        else:
            occurences[factorOfStudent] = 1
    return occurences

In [13]:
testGroupData = [
    {"Student ID": 71, "TG" : "G-1", "School" : "SoH", "Name" : "Savannah Taylor", "Gender" : False, "CGPA" : 4.19},
    {"Student ID": 75, "TG" : "G-1", "School" : "CCDS", "Name" : "Sakina Ahmed", "Gender" : False, "CGPA" : 4.03},
    {"Student ID": 162, "TG" : "G-1", "School" : "SSS", "Name" : "Lila Patel", "Gender" : False, "CGPA" : 4.07},
    {"Student ID": 235, "TG" : "G-1", "School" : "CCDS", "Name" : "Ming Zhang", "Gender" : True, "CGPA" : 4.06},
    {"Student ID": 288, "TG" : "G-1", "School" : "CoB (NBS)", "Name" : "Ajay Verma", "Gender" : True, "CGPA" : 4.01}
]
print(FindOccurencesInGroup("School", testGroupData))
print(FindOccurencesInGroup("Gender", testGroupData))
print(FindOccurencesInGroup("CGPA", testGroupData))

{'SoH': 1, 'CCDS': 2, 'SSS': 1, 'CoB (NBS)': 1}
{False: 3, True: 2}
{4.19: 1, 4.03: 1, 4.07: 1, 4.06: 1, 4.01: 1}


#### Then find the most number of occurences out of the total students

In [14]:
# Summary: returns the key and value of the key with the largest value
# Parameters:
#  dictionary: the dict to find the max of
# Return:
#  key : list, values : int, where key is the key with highest value
def FindMaxInDict(dictionary: dict) -> (list, int):
    maxKeys = []
    maxVal = 0
    for key, value in dictionary.items():
        # update maxKeys and maxVal whenever a new value >= current maxVal
        if value > maxVal:
            maxKeys = [key]
            maxVal = value
        elif value == maxVal:
            maxKeys.append(key)
    return maxKeys, maxVal

In [15]:
print(FindMaxInDict(FindOccurencesInGroup("School", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("Gender", testGroupData)))
print(FindMaxInDict(FindOccurencesInGroup("CGPA", testGroupData)))

(['CCDS'], 2)
([False], 3)
([4.19, 4.03, 4.07, 4.06, 4.01], 1)


## CGPA case
But this does not work for CGPA, for that we want to find the mean, range, and midrange instead
If we use only mean, a majority of high CGPA with a very low CGPA will result in a medium CGPA, this can be solved by looking at midrange, where a high difference between midrange and mean will show that this is happening
However, even with these two, a group will all medium CGPAs will also result in medium mean, and low midrange difference, therefore we add range to determine that this is happening if the range is small.
In conclusion, the result we are looking for is one with an medium mean, high range, and low difference between midrange and mean.
### Finding mean, midrange, and range

In [56]:
def Mean(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return sum(nums) / len(nums)

def Range(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return max(nums) - min(nums)

def Midrange(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    return (max(nums) + min(nums)) / 2

def Median(*nums) -> float:
    if len(nums) == 1 and type(nums[0]) is list:
        nums = nums[0]
    # TODO: replace with another self-made sort cherry pls
    nums.sort()
    total = len(nums)
    mid = total // 2
    if total % 2:
        return (nums[mid] + nums[mid+1]) / 2
    return nums[mid]

In [57]:
testList = [4.19, 4.03, 4.07, 4.06, 4.01]
print(Mean(testList))
print(Range(testList))
print(Midrange(testList))
print(Median(testList))

4.072
0.1800000000000006
4.1
4.0649999999999995


## Calculating for CGPA

In [121]:
# Parameters:
#  medianCGPA: median CGPA to use for calculation (likely the tutorial grp's)
#  CGPARange: Range of CGPA to use for calculation (likely the tutorial grp's)
#  groupData: list of studentData to find CGPA diversity
# Return:
#  score: overall score of CGPA
#  cGPADetails: dictionary of {factor: percent} the factor "balance" students with higher CGPA - students with lower CGPA than median
def CalculateCGPADiversity(medianCGPA : float, CGPARange : float, *groupData) -> (int, dict):
    if len(groupData) == 1 and type(groupData[0]) is list:
        groupData = groupData[0]

    cGPADetails = {}
    # Calculate higher and lowers
    lower = higher = 0
    cGPAs = ExtractCGPA(groupData)
    for gpa in cGPAs:
        if gpa < medianCGPA:
            lower += 1
        elif gpa > medianCGPA:
            higher += 1
    cGPADetails["balance"] = higher - lower

    # Calculate the score for each factor
    cGPADetails["mean"] = Mean(cGPAs)
    cGPADetails["range"] = Range(cGPAs)
    cGPADetails["midrange"] = Midrange(cGPAs)

    # Score 0 to 100, 100 is most diverse possible
    # Usage of 5 as is prevalent as it is the max CGPA and max difference between lowest and highest CGPA
    
    # vv How close mean is to the median vv
    cGPADetails["meanScore"] = (5 - abs(cGPADetails["mean"] - medianCGPA)) * 20
    # vv Ratio of grp's range and tutorial grp's range vv
    cGPADetails["rangeScore"] = cGPADetails["range"] / CGPARange * 100
    # vv How far midrange is from mean vv   most diverse ==> mean of x, midrange of x
    cGPADetails["midrangeScore"] = (1 - abs(cGPADetails["mean"] - cGPADetails["midrange"])) * 100
    # vv Lowest: score = 0 when abs(balance) = Num of grp members, Highest: score = 100 when balance = 0 vv 
    cGPADetails["balanceScore"] = (len(groupData) - abs(cGPADetails["balance"])) / len(groupData) * 100
    
    # Overall score out of 100
    # Weightages for what we consider more important
    W_RANGE = 1
    W_MIDRANGE = 1
    W_MEAN = 2
    W_BALANCE = 2
    
    score = (cGPADetails["meanScore"] * W_MEAN\
             + cGPADetails["rangeScore"] * W_RANGE\
             + cGPADetails["midrangeScore"] * W_MIDRANGE\
             + cGPADetails["balanceScore"] * W_BALANCE) / sum([W_MEAN, W_RANGE, W_MIDRANGE, W_BALANCE])
    return score, cGPADetails

# Returns a list of CGPA of the students in groupData
def ExtractCGPA(*groupData) -> list:
    if len(groupData) == 1 and type(groupData[0]) is list:
        groupData = groupData[0]

    cGPAs = []
    for i in groupData:
        cGPAs.append(i["CGPA"])
    return cGPAs

# Returns a list of CGPA of the students in groupData
def TutGrpExtractCGPA(tutGroupData) -> list:
    cGPAs = []
    for i in tutGroupData:
        cGPAs.extend(ExtractCGPA(i))
    return cGPAs

In [122]:
#TODO : revise the calculation of balance factor by taking into account buckets rather than median
testGroupData = finalGrps
medianCGPA = Median(TutGrpExtractCGPA(testGroupData))
rangeCGPA = Range(TutGrpExtractCGPA(testGroupData))
print(medianCGPA)
for i in range(len(testGroupData)):
    print(f"test group: {i}")
    PrintDictListNice(testGroupData[i])
    print()
    result = CalculateCGPADiversity(medianCGPA, rangeCGPA, testGroupData[i])
    print("Score: ", result[0])
    PrintDictListNice([result[1]])
# print(testGroupData)


4.1
test group: 0
ID: 3721              TG: 120               School: CoB (NBS)         Name: Samantha Lee      Gender: False             CGPA: 4.520-            
ID: 5220              TG: 120               School: CoB (NBS)         Name: Rishi Singh       Gender: True              CGPA: 4.080-            
ID: 3222              TG: 120               School: CCDS              Name: Kang Lei          Gender: False             CGPA: 4.040-            
ID: 5835              TG: 120               School: MAE               Name: Samuel Cook       Gender: True              CGPA: 4.020-            
ID: 822               TG: 120               School: CCEB              Name: Riku Tanaka       Gender: False             CGPA: 4.030-            
ID: 4631              TG: 120               School: EEE               Name: Samantha Patel    Gender: False             CGPA: 4.000-            
ID: 2420              TG: 120               School: CEE               Name: Faisal Bin Yusof  Gender: True      

## Calculating the factors for each group

In [27]:
# Parameters:
#  factors: list of strings of factor to check, same as the key in the dictionary
#  groupsDatas: List of datas needed as a list of dictionaries
# Return:
#  leastDiverse: dictionary of {factor1: occurencesPercent, factor2: occurencesPercent ... } as string:int
#  grpIdx: dictionary of the grps that have the least diversity for the factor {factor1: groups}
def FindLeastDiverseInTutorialGroup(factors, *groupsDatas) -> (dict, list):
    if len(groupsDatas) == 1 and type(groupsDatas[0]) is list:
        groupsDatas = groupsDatas[0]

    grpIdx = {}
    leastDiverse = {}
    for factor in factors:
        # TODO: Special CGPA case
        
        # Set default value to replace
        leastDiverseOccurence : float = ("factorResult", 0) # % of highest occurence, i.e. 90% of students are from ccds
        currentGrp = 0
        for i in groupsDatas:
            occurence = FindMaxInDict(FindOccurencesInGroup(factor, i))
            currentPercent = occurence[1] / len(i)
            # Less diverse so replace
            if currentPercent > leastDiverseOccurence[1]:
                leastDiverseOccurence = (occurence[0], currentPercent)
                grpIdx[factor] = [currentGrp]
            # Same diversity so place
            elif currentPercent == leastDiverseOccurence[1]:
                leastDiverseOccurence = (list(leastDiverseOccurence[0] + occurence[0]), currentPercent)
                grpIdx[factor].append(currentGrp)
            currentGrp += 1
        leastDiverse[factor] = leastDiverseOccurence
    return leastDiverse, grpIdx

In [48]:
testGroupData = finalGrps
for i in range(len(testGroupData)):
    print(f"test group: {i}")
    PrintDictListNice(testGroupData[i])
    print()
# print(testGroupData)
PrintDictListNice(FindLeastDiverseInTutorialGroup(["School", "Gender", "CGPA"], testGroupData))

test group: 0
ID: 3721              TG: 120               School: CoB (NBS)         Name: Samantha Lee      Gender: False             CGPA: 4.52              
ID: 5220              TG: 120               School: CoB (NBS)         Name: Rishi Singh       Gender: True              CGPA: 4.08              
ID: 3222              TG: 120               School: CCDS              Name: Kang Lei          Gender: False             CGPA: 4.04              
ID: 5835              TG: 120               School: MAE               Name: Samuel Cook       Gender: True              CGPA: 4.02              
ID: 822               TG: 120               School: CCEB              Name: Riku Tanaka       Gender: False             CGPA: 4.03              
ID: 4631              TG: 120               School: EEE               Name: Samantha Patel    Gender: False             CGPA: 4.0               
ID: 2420              TG: 120               School: CEE               Name: Faisal Bin Yusof  Gender: True          

In [24]:
# Weights
CRITERA_WEIGHTS = {"CGPA": 100, "SCHOOL": 100, "GENDER": 100}

# Graphical plots and things

# Challenges and Issues

- 

# Conclusion

# Debug test things or test cases?
doubt they will give marks for test cases tho, prob just leave this for our own use and remove before submission
Highlight

In [94]:
def PrintDictListNice(dictionaryList, spacing : int = 18):
    for i in dictionaryList:
        for k,v in i.items():
            contentLength = 10
            if type(v) is float:
                v = f"{v:.3f}-"
            if type(v) is int:
                contentLength = log10(v) + 1
            else:
                contentLength = len(str(v))
            # elif type(v) is bool:
            #     contentLength = len(str(v))
            print(f"{k}: {v}{' ' * max(spacing - contentLength, 1)}", end = '')
        print()
def log10(num):
    count = 0
    while num >= 10:
        num /= 10
        count += 1
    return count

In [95]:
print(log10(1))
print(log10(20))
print(log10(34))
print(log10(64280))
d = {"test" : 4.236752387546, "a" : "g"}
PrintDictListNice([d])

0
1
1
4
test: 4.237-            a: g                 
